In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date, timedelta
from matplotlib import style
pd.core.common.is_list_like = pd.api.types.is_list_like
import fix_yahoo_finance as yf
from pandas_datareader import data as pdr

In [2]:
ticker = ['SAN.MC','TEF.MC','SGRE.MC','ENG.MC','ITX.MC']
allocations = [20,20,20,20,20] # numero de acciones que colocaremos 
buy_in_date = '2017-08-28'


In [3]:
fecha_inicio= buy_in_date
fecha_final= buy_in_date

In [4]:

yf.pdr_override() 
price_data_today = pdr.get_data_yahoo(ticker, 
                           fecha_inicio,
                            fecha_final)['Adj Close']

[*********************100%***********************]  5 of 5 downloaded


In [5]:
allocation_df = pd.DataFrame({'Symbol': ticker, 'Allocations': allocations}).set_index('Symbol')

equity_positions = price_data_today * allocations
port_val_today = equity_positions.sum().sum()
weights = equity_positions /port_val_today
price_data = price_data_today.transpose()





# REBALANCEO

In [6]:
#########################################################################################################################
buy_in_date = dt.datetime(2018, 8,28)
price_at_buy_in = pdr.get_data_yahoo(ticker, 
                           buy_in_date,
                            buy_in_date)['Adj Close']

[*********************100%***********************]  5 of 5 downloaded


In [7]:
equity_positions_prior = price_at_buy_in * allocations     #mutiplication de price por allocations (numero de acciones)
equity_positons_prior = equity_positions_prior.transpose() # lo transformo en un vector

port_value_prior =np.sum(equity_positions_prior).sum()


weights_prior = equity_positions_prior / port_value_prior

In [9]:
weights.transpose()

Date,2017-08-28 00:00:00
ENG.MC,0.283291
ITX.MC,0.397442
SAN.MC,0.063508
SGRE.MC,0.152731
TEF.MC,0.103028


In [24]:
result = pd.concat([allocation_df, price_data, weights.transpose(), weights_prior.transpose() ], axis = 1)
result.columns = ['original num shares', 'current share price', 'current weights', 'weights prior']
result['Status'] = np.where(result['current weights'] >result['weights prior'], 'Overweight', 'Underweight' )
result

C:\Users\abelb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,original num shares,current share price,current weights,weights prior,Status
ENG.MC,20,23.064991,0.283291,0.311534,Underweight
ITX.MC,20,32.358917,0.397442,0.367527,Overweight
SAN.MC,20,5.170694,0.063508,0.056668,Overweight
SGRE.MC,20,12.435000,0.152731,0.169863,Underweight
TEF.MC,20,8.388310,0.103028,0.094409,Overweight


In [25]:
solve_weights = []
for r in range(0,5):
    x = 0.00
    al = result.iloc[r,0] # numero de acciones
    sp = result.iloc[r, 1] # stock price
    eq = al * sp # posicion del capital
    cw = eq /port_val_today # current weight
    status = result.iloc[r, 4] # over / under
    dw = result.iloc[r,3] # peso deseado
    
    while True:
        e = x * sp
        w = e/port_val_today
        x+= 0.01
        
        if status == 'Overweight':
            if w >= dw:
                solve_weights.append(int(x))
                break
                
        elif status == 'Underweight':
            if w >= dw:
                solve_weights.append(int(x))
                break
        else:
            break

In [26]:
solve_weights = pd.DataFrame({'Symbol':ticker,
                             'New Share Amount':solve_weights}).set_index('Symbol')
result = pd.concat([result , solve_weights], axis = 1)

result['# Buy/Sell'] = np.where(result['Status']=='Overweight',
                                result['original num shares'] -result['New Share Amount'],
                               result['New Share Amount'] -  result['original num shares'])

result['Action']= np.where(result['Status']== 'Overweight','Sell','Buy')

print('Number of Shares to Buy/Sell')
pd.concat([result['Action'], result['# Buy/Sell']], axis = 1)



Number of Shares to Buy/Sell


C:\Users\abelb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


,Action,# Buy/Sell
ENG.MC,Buy,2
ITX.MC,Sell,2
SAN.MC,Sell,-2
SGRE.MC,Buy,-3
TEF.MC,Sell,2
